#Instalação 

In [ ]:
# reading daaset from Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install emoji
!pip install simplejson
!pip install wmd
!pip install kneed

#BERTopic
!pip install bertopic[all]
!pip install umap-learn

#Metrics
!pip install octis
!pip install tmplot
!pip install git+https://github.com/maximtrp/tmplot.git
!pip install -U contextualized_topic_models

#embedding
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install pyyaml==5.4.1
!pip install flair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 197 kB 15.4 MB/s 
  Created wheel for emoji: filename=emoji-2.0.0-py3-none-any.whl size=193022 sha256=1ca35b9fa522eadb48254b65d40742e41483b2f9a4ca1907a2312a4c497d6995
  Stored in directory: /root/.cache/pip/wheels/ec/29/4d/3cfe7452ac7d8d83b1930f8a6205c3c9649b24e80f9029fc38
Successfully built emoji
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 130 kB 37.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 637 kB 24.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76 kB 5.1 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2022-09-04 16:32:46.149707: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 28.9 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 401 kB 30.7 MB/s 
     |████████████████████████████████| 788 kB 48.8 MB/s 
     |████████████████████████████████| 1.2 MB 60.3 MB/s 
     |████████████████████████████████| 981 kB 68.7 MB/s 
     |████████████████████████████████| 1.6 MB 50.

In [ ]:
 # Packages to store and manipulate data
import pandas as pd
import numpy as np
# Example function using numpy:
from numpy import dot
from numpy.linalg import norm
import math
import os
import json
import simplejson as json
import csv
import string
import glob
import random
import time
import math
from datetime import datetime
pd.set_option('max_colwidth', 200)
 
# Plotting packages
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
import pickle
import math
import wmd
from scipy import spatial
from scipy.spatial import distance
from scipy.spatial.distance import pdist, squareform
from collections import Counter
%matplotlib inline
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import tmplot as tmp
from tmplot._helpers import get_phi
from tmplot._helpers import calc_terms_probs_ratio
import umap
from umap import UMAP
 
# Packages
import emoji
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
import operator
from operator import itemgetter
from tqdm import tqdm
from collections import defaultdict
from textblob import TextBlob
import sys
import itertools
import spacy
from scipy.sparse import csr_matrix, issparse  # , todense
import sys
from collections import defaultdict
import multiprocessing
from pprint import pprint
from contextualized_topic_models.evaluation.measures import CoherenceNPMI
from hdbscan import HDBSCAN
 
# Gensim
import gensim
import gensim.downloader
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
from gensim import models
from gensim import corpora
from gensim.models import CoherenceModel
import gensim.downloader as api
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.models.ldamulticore import LdaMulticore
 
# Model building package: Sklearn
import sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.datasets import make_multilabel_classification
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import cosine_distances
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import pairwise_kernels
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE
from sklearn.datasets import load_digits
# Cluster documents: Sklearn
from kneed import KneeLocator
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
 
#BERTopic
#Embeddings
from bertopic import BERTopic
from bertopic.backend import WordDocEmbedder
from sentence_transformers import SentenceTransformer
from flair.embeddings import TransformerDocumentEmbeddings
from flair.embeddings import TransformerWordEmbeddings

#Metrics
from octis.evaluation_metrics.metrics import AbstractMetric
from octis.dataset.dataset import Dataset
from gensim.corpora.dictionary import Dictionary
from gensim.models import CoherenceModel
import octis.configuration.citations as citations
import numpy as np
import itertools
from scipy import spatial
from sklearn.metrics import pairwise_distances
from operator import add
from octis.evaluation_metrics.rbo import rbo
import gensim.downloader as api
from gensim.models import KeyedVectors
from octis.evaluation_metrics.diversity_metrics import WordEmbeddingsInvertedRBO, \
    WordEmbeddingsInvertedRBOCentroid, InvertedRBO
from itertools import combinations
from scipy.spatial.distance import cosine
import gensim.downloader as api

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


#Importar Dados

In [ ]:
df = pd.read_json('/content/drive/dados_fevereiro_dia_de_semana_1.json')

#Tokenizar
new_tweet = []
for tweet in df.clean_tweet:
    tweet_len = [word for word in tweet.split(' ') if len(word) >= 3]
    tweet_string = ' '.join(tweet_len)
    new_tweet.append(tweet_string)
df['clean_tweet'] = new_tweet

stop = stopwords.words('english')
df['clean_tweet_tokenize'] = df['clean_tweet'].apply(lambda x: [item for item in str(x).split(' ')])
df.dropna()

,created_at,id,tweet,clean_tweet,clean_tweet_tokenize
0,2022-02-07 00:52:56,1490488714312785920,Winters in Florida…toe deep in snow white sand! #ilivewhereyouvacation #emeraldcoast @OkaloosaCounty @CityOfFWB @VISITFLORIDA https://t.co/vwwlH9pipN,winters florida toe deep snow white sand,"[winters, florida, toe, deep, snow, white, sand]"
1,2022-02-07 00:52:52,1490488698810425344,Snow much fun ❄️🏂 https://t.co/7E59fOOAC2,snow fun,"[snow, fun]"
2,2022-02-07 00:51:46,1490488422548443136,Grateful To Be Alive ❤️❤️🙏🏾\nHappy Sunday Everyone ☀️\n\n#sunday #sundayvibes #sunny #la #fyp #explore #fypシ #explorepage #blue #bluehair #skin #skincare #glow #glowup #blessed #home #glasses #lov...,grateful alive happy sunday,"[grateful, alive, happy, sunday]"
3,2022-02-07 00:50:02,1490487984218591232,48.6F (Feels: 47.2F) - Humidity: 66% - Wind: 0.0mph NE - Gust: 0.0mph - Pressure: 30.161in #weather https://t.co/75yzNJ23Eh,feels humidity wind mph gust mph pressure,"[feels, humidity, wind, mph, gust, mph, pressure]"
4,2022-02-07 00:49:08,1490487759575867392,Festive snow https://t.co/HsBLRkIgR3,festive snow,"[festive, snow]"
...,...,...,...,...,...
63112,2022-02-11 23:02:06,1492272762202857472,FORMER BREITBART REPORTER GETTING DEATH THREATS FOR REVEALING OBAMA’S FAKE PAST! https://t.co/XvKj8P1Jq4 # via @AFF_PATRIOTS cc @javelini11,breitbart reporter getting death threats revealing obama fake past,"[breitbart, reporter, getting, death, threats, revealing, obama, fake, past]"
63113,2022-02-11 23:01:46,1492272679273091072,"Today I thought about a memory with my old man and actually laughed out loud instead of getting that sick feeling in my heart…and that’s a first. \n\nI’ll still be here stargazing, but the tides i...",today thought memory old man actually laughed loud instead getting sick feeling heart stargazing tides soul finally starting turn thankful,"[today, thought, memory, old, man, actually, laughed, loud, instead, getting, sick, feeling, heart, stargazing, tides, soul, finally, starting, turn, thankful]"
63114,2022-02-11 23:01:44,1492272670100140032,I'm currently sitting in the waiting room at the hospital reluctantly af but I'm in relapse with my MS and don't have a choice. Waiting for a CT scan and MRI so they can diagnose the optic neuriti...,currently sitting waiting room hospital reluctantly relapse choice waiting scan mri diagnose optic neuritis nice course steroids,"[currently, sitting, waiting, room, hospital, reluctantly, relapse, choice, waiting, scan, mri, diagnose, optic, neuritis, nice, course, steroids]"
63115,2022-02-11 23:01:32,1492272622230548480,"It’s been a really nice day. Everyone is so lovely@and supportive of the new release. Kinda sucks since I’m currently sick with Covid, but y’all are making it a great day ❤️",nice day lovely supportive release kinda sucks currently sick covid making great day,"[nice, day, lovely, supportive, release, kinda, sucks, currently, sick, covid, making, great, day]"


#Modelagem de Tópicos

In [ ]:
# cast tweets to numpy array
docs = df.clean_tweet_tokenize.to_numpy()

# create dictionary of all words in all documents
dictionary =  corpora.Dictionary(docs)

# create variable containing length of dictionary/vocab
vocab_length = len(dictionary)

# create BOW dictionary
bow_corpus = [dictionary.doc2bow(doc) for doc in docs]

In [ ]:
#UMAP
umap_model = UMAP(n_neighbors=15, n_components=5, 
                  min_dist=0.0, metric='cosine', random_state=42)

In [ ]:
# Word embedding
embedding_word_2 = TransformerWordEmbeddings('roberta-large')

# Document embedding model
embedding_document_4 = SentenceTransformer('all-mpnet-base-v2')

# Create a model that uses both language models and pass it through BERTopic
word_doc_embedder = WordDocEmbedder(embedding_model=embedding_document_4, word_embedding_model=embedding_word_2)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
topic_model = BERTopic(nr_topics= "auto", language='english', calculate_probabilities=True, embedding_model=word_doc_embedder, umap_model=umap_model, verbose=True).fit(list(df.clean_tweet))
topics, probabilities = topic_model.fit_transform(list(df.clean_tweet))

Batches:   0%|          | 0/1970 [00:00<?, ?it/s]

2022-09-04 18:47:54,340 - BERTopic - Transformed documents to Embeddings
2022-09-04 18:49:56,116 - BERTopic - Reduced dimensionality
2022-09-04 19:09:17,061 - BERTopic - Clustered reduced embeddings
2022-09-04 19:58:27,469 - BERTopic - Reduced number of topics from 617 to 363


Batches:   0%|          | 0/1970 [00:00<?, ?it/s]

2022-09-04 22:16:54,023 - BERTopic - Transformed documents to Embeddings
2022-09-04 22:18:43,033 - BERTopic - Reduced dimensionality
2022-09-04 22:39:41,209 - BERTopic - Clustered reduced embeddings
2022-09-04 23:28:48,887 - BERTopic - Reduced number of topics from 617 to 363


In [ ]:
# Save model
topic_model.save("model_train")	
# Load model
topic_model_test = BERTopic.load("model_train")	

In [ ]:
nr_topics = 10
new_topics, new_probs = topic_model.reduce_topics(df.clean_tweet, topics, probabilities, nr_topics=nr_topics)
freq = topic_model.get_topic_info();

2022-09-04 23:29:25,926 - BERTopic - Reduced number of topics from 363 to 11


#Tópicos

In [ ]:
topic = pd.DataFrame()
topics_train = []
for i in range(1, nr_topics + 1):
  top = []
  name = 'Topic '+ str(i - 1)
  topic_nr = freq.iloc[i]["Topic"]
  for j in range(len(topic_model.get_topic(topic_nr))):
    top.append(topic_model.get_topic(topic_nr)[j][0])
  topics_train.append(top)
  topic[name] = top
topic 

,Topic 0,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7,Topic 8,Topic 9
0,mph,wordle,relationship,spotify,olympic,sunny,ukraine,oscar,chocolate,pizza
1,wind,streak,dating,playlist,olympics,beach,russia,awards,cake,national
2,humidity,close,boyfriend,podcast,winter,cloudy,putin,vote,strawberry,day
3,rain,guess,girlfriend,episode,figure,isles,biden,voting,strawberries,happy
4,feels,phew,like,rogan,skating,windows,russian,actor,covered,pineapple
5,day,squares,want,music,gold,weather,war,nominations,valentine,slice
6,today,word,partner,apple,games,degrees,invade,zendaya,cream,today
7,gust,nyt,tinder,joe,hockey,partly,invasion,nomination,day,best
8,pressure,letters,love,listen,sport,day,nato,best,vanilla,favorite
9,inch,letter,apps,songs,watching,snow,troops,nominated,cookie,celebrate


#Métricas

In [ ]:
cm_bertopic = CoherenceModel(topics=topics_train, 
                          dictionary=dictionary, 
                          corpus=bow_corpus, 
                          texts=docs, 
                          coherence='c_v')

print(cm_bertopic.get_coherence()) 

0.5937387000751514


In [ ]:
npmi = CoherenceModel(topics=topics_train, 
                          dictionary=dictionary, 
                          corpus=bow_corpus, 
                          texts=docs,
                          coherence='c_npmi')

print(npmi.get_coherence())

0.12922344201462507


In [ ]:
def get_word2index(list1, list2):
    words = set(list1)
    words = words.union(set(list2))
    word2index = {w: i for i, w in enumerate(words)}
    return word2index

In [ ]:
#Diversity Metrics:
class TopicDiversity(AbstractMetric):
    def __init__(self, topk=10):
        """
        Initialize metric
        Parameters
        ----------
        topk: top k words on which the topic diversity will be computed
        """
        AbstractMetric.__init__(self)
        self.topk = topk

    def score(self, model_output):
        """
        Retrieves the score of the metric
        Parameters
        ----------
        model_output : dictionary, output of the model
                       key 'topics' required.
        Returns
        -------
        td : score
        """
        topics = model_output
        if topics is None:
            return 0
        if self.topk > len(topics[0]):
            raise Exception('Words in topics are less than ' + str(self.topk))
        else:
            unique_words = set()
            for topic in topics:
                unique_words = unique_words.union(set(topic[:self.topk]))
            td = len(unique_words) / (self.topk * len(topics))
            return td

In [ ]:
class InvertedRBO(AbstractMetric):
    def __init__(self, topk=10, weight=0.9):
        """
        Initialize metric Inverted Ranked-Biased Overlap
        :param topk: top k words on which the topic diversity will be computed
        :param weight: weight of each agreement at depth d. When set to 1.0, there is no weight, the rbo returns to
        average overlap. (default 0.9)
        """
        super().__init__()
        self.topk = topk
        self.weight = weight

    def score(self, model_output):
        """
        Retrieves the score of the metric
        :param model_output : dictionary, output of the model. the 'topics' key is required.
        """
        topics = model_output
        if topics is None:
            return 0
        if self.topk > len(topics[0]):
            raise Exception('Words in topics are less than topk')
        else:
            collect = []
            for list1, list2 in itertools.combinations(topics, 2):
                word2index = get_word2index(list1, list2)
                indexed_list1 = [word2index[word] for word in list1]
                indexed_list2 = [word2index[word] for word in list2]
                rbo_val = rbo(indexed_list1[:self.topk], indexed_list2[:self.topk], p=self.weight)[2]
                collect.append(rbo_val)
            return 1 - np.mean(collect)

In [ ]:
#Similarity Metrics:
class PairwiseJaccardSimilarity(AbstractMetric):
    def __init__(self, topk=10):
        """
        Initialize metric Pairwise Jaccard Similarity
        Parameters
        ----------
        :param topk: top k words on which the topic diversity will be computed
        """
        super().__init__()
        self.topk = topk

    def score(self, model_output):
        """
        Retrieves the score of the metric
        :return PJS
        """
        topics = model_output
        sim = 0
        count = 0
        for list1, list2 in combinations(topics, 2):
            intersection = len(list(set(list1[:self.topk]).intersection(list2[:self.topk])))
            union = (len(list1[:self.topk]) + len(list2[:self.topk])) - intersection
            count = count + 1
            sim = sim + (float(intersection) / union)
        return sim / count

In [ ]:
diversity = TopicDiversity()
diversity.score(topics_train)

0.95

In [ ]:
b = InvertedRBO()
b.score(topics_train)

0.9911746551828572

In [ ]:
jaccard = PairwiseJaccardSimilarity()
jaccard.score(topics_train)

0.009486679662118257

##Visualization

In [ ]:
representative_docs = topic_model.get_representative_docs(5)
representative_docs #Para obter os documentos representativos de um único tópico

['try enjoy sunshine warm air february today saturday big cooldown coming snowflakes',
 'degrees willows amsinckia flowering uncomfortably warm february little rain winter worried going short hot spring long hot summer',
 'better sunny day',
 'city sunny isles beach',
 'city sunny isles beach',
 'cecile sippin beach sunny isles beach',
 'forecast intermittent clouds partly sunny sunny sunny sunny sunny sunny sunny',
 'forecast intermittent clouds partly sunny sunny sunny sunny sunny sunny sunny',
 'forecast clear clear clear clear clear partly cloudy partly cloudy partly cloudy',
 'minneapolis great cloudy',
 'minneapolis pete sake partly cloudy',
 'minneapolis forecast today cloudy day fantastic',
 'great scott partly cloudy',
 'golly gee wilikers cloudy',
 'great cloudy',
 'short range models excited denver snow chances tomorrow afternoon think main impacts foothills moisture lacking temps borderline surprised maybe denver colder surfaces',
 'little rain snow mix moisture day october

In [ ]:
fig = topic_model.visualize_topics(); fig

In [ ]:
topics_over_time = topic_model.topics_over_time(docs=df.clean_tweet, 
                                                topics=new_topics, 
                                                timestamps=df.created_at, 
                                                global_tuning=True, 
                                                evolution_tuning=True, 
                                                nr_bins=20)

20it [00:04,  4.14it/s]


In [ ]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=8)

In [ ]:
topic_model.visualize_hierarchy(top_n_topics=11)

In [ ]:
topic_model.visualize_barchart(top_n_topics=11)